In [1]:
import findspark
findspark.init()
findspark.find()

'C:\\spark\\spark-3.4.1-bin-hadoop3'

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.types import StructType, StructField, StringType, FloatType

In [3]:
from datetime import datetime

In [4]:
#time module required to capture time and measure performance
import time 
start_time = time.time()

In [5]:
#Initiate Spark Session
spark = SparkSession \
    .builder \
    .appName("Covid Time Series Analysis") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [6]:
#read CSV
df = spark.read.options(header=True, inferSchema = True).csv('time_series_covid19_confirmed_global.csv')

In [7]:
#rename columns
df = df.withColumnRenamed("Country/Region", "Country")
df = df.withColumnRenamed("Province/State", "State")

#drop Columns
drop_pos_columns = ("Lat", "Long")
df = df.drop(*drop_pos_columns)

#Exclude Outliers
excluded_countries = ["Diamond Princess", "MS Zaandam", "Summer Olympics 2020", "Winter Olympics 2022"]
df = df.filter(~F.col("Country").isin(excluded_countries))

In [8]:
df.show()

+--------------------+-------------------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+------+------+------+------+------+------+------+------+------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+------+------+------+------+------+------+------+------+------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+------+------+------+------+------+------+------+------+------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+------+------+------+------+------+------+------+------+------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+--

In [9]:
#Creating New column Zone
df2 = df.withColumn("Zone", \
                         F.when(F.isnull(df.State), df.Country) \
                         .otherwise(df.State)
                         )

In [10]:
#pivot date and daily cases
id_vars = ["State", "Country", "Zone"]
value_vars = [f'"{c}",`{c}`' for c in df.columns if c not in id_vars ]

stack_expr = F.expr(f"stack({len(value_vars)}, {', '.join(value_vars)}) as (date, value)")
df2_reshaped = df2.select(id_vars + [stack_expr])

#change the data type of Date column from string to date format
df2_reshaped = df2_reshaped.withColumn("date", F.to_date("date","M/d/yy"))

In [11]:
#Adding a column No_of_days that signify the total no of days elapsed from the start
min_date = df2_reshaped.select(F.min("date")).collect()[0][0]

df2_reshaped= df2_reshaped.withColumn("No_of_Days", F.datediff(F.col('date'), F.lit(min_date)))

In [12]:
#Calculate daily increases by implementing current date-previus date
window_spec = Window.partitionBy("Zone").orderBy("date")
df_window = df2_reshaped.orderBy("Country").withColumn("daily_increases", F.col("value") - F.lag("value").over(window_spec))

In [13]:
#dealing with the null values in daily_increases
df_window = df_window.withColumn("daily_increases", \
                         F.when(F.isnull(df_window.daily_increases), df_window.value) \
                         .otherwise(df_window.daily_increases)
                         ) 

In [14]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import StandardScaler 
from pyspark.ml.clustering import KMeans 
from pyspark.ml.evaluation import ClusteringEvaluator 

In [15]:
vector_assembler1 = VectorAssembler(inputCols=["No_of_Days"], outputCol="features")

lr = LinearRegression(featuresCol='features', labelCol = 'label') 

In [16]:
#creating list of zones to iterate through
zones = df_window.select("Zone").distinct().collect()

print(zones)

[Row(Zone='Chad'), Row(Zone='Manitoba'), Row(Zone='Paraguay'), Row(Zone='Russia'), Row(Zone='Anguilla'), Row(Zone='Yemen'), Row(Zone='Guangdong'), Row(Zone='Korea, North'), Row(Zone='Senegal'), Row(Zone='Cabo Verde'), Row(Zone='Sweden'), Row(Zone='Kiribati'), Row(Zone='Hunan'), Row(Zone='Shanxi'), Row(Zone='Guyana'), Row(Zone='Burma'), Row(Zone='Tibet'), Row(Zone='Eritrea'), Row(Zone='Philippines'), Row(Zone='Jersey'), Row(Zone='Djibouti'), Row(Zone='Tonga'), Row(Zone='Malaysia'), Row(Zone='Singapore'), Row(Zone='Hubei'), Row(Zone='Fiji'), Row(Zone='Turkey'), Row(Zone='Yukon'), Row(Zone='Tianjin'), Row(Zone='Malawi'), Row(Zone='Northern Territory'), Row(Zone='Beijing'), Row(Zone='Iraq'), Row(Zone='Germany'), Row(Zone='Comoros'), Row(Zone='Afghanistan'), Row(Zone='Cambodia'), Row(Zone='Jordan'), Row(Zone='Maldives'), Row(Zone='Rwanda'), Row(Zone='Sudan'), Row(Zone='Palau'), Row(Zone='France'), Row(Zone='Turks and Caicos Islands'), Row(Zone='Greece'), Row(Zone='Kosovo'), Row(Zone='Sri La

In [17]:
result = []

In [18]:
#iterating through each zone and calculating the trendline coefficient
for zone in zones:
    df_zone = df_window.filter(df_window['Zone'] == zone[0]).select('No_of_Days','daily_increases')
    df_zone = df_zone.withColumnRenamed('daily_increases','label')
    df_zone_vector = vector_assembler1.transform(df_zone)
    model = lr.fit(df_zone_vector)
    trendline_coefficient = float(model.coefficients[0])
    result.append([zone [0], trendline_coefficient])

print(result)

[['Chad', -0.006532846425561239], ['Manitoba', 0.05721877512974767], ['Paraguay', 0.2240196760550598], ['Russia', 16.464437211137984], ['Anguilla', 0.005211799384200336], ['Yemen', -0.008412905543923103], ['Guangdong', 0.3686350147042123], ['Korea, North', 2.1858068936684977e-06], ['Senegal', -0.07072891806126784], ['Cabo Verde', -0.01094125730762666], ['Sweden', 0.5866950480834147], ['Kiribati', 0.011824797419899256], ['Hunan', 0.01974093013090765], ['Shanxi', 0.024976339443981508], ['Guyana', 0.03526536643943595], ['Burma', -0.0968252183484471], ['Tibet', 0.005224777612631496], ['Eritrea', -0.003273036885844923], ['Philippines', 0.40715207426805927], ['Jersey', 0.09750150057250453], ['Djibouti', -0.022944985522725926], ['Tonga', 0.037288033385500186], ['Malaysia', 4.109667476733129], ['Singapore', 4.460320854593922], ['Hubei', -0.28688951739408874], ['Fiji', 0.021016043083282452], ['Turkey', 7.294280976090078], ['Yukon', 0.006095460037294371], ['Tianjin', 0.010284623237448089], ['Mal

In [19]:
#converting list to dataframe
from pyspark.sql import Row

R = Row("Zone", "trendline_coefficient")
df_result = spark.createDataFrame([R(*i) for i in result])

df_result = df_result.orderBy(F.desc("trendline_coefficient"))
df_top_50 = df_result.limit(50)

df_top_50.show(50)

+-----------------+---------------------+
|             Zone|trendline_coefficient|
+-----------------+---------------------+
|            Japan|    94.19123188154933|
|     Korea, South|    72.67960297903623|
|          Germany|    65.83472445477216|
|           France|    52.23886925252118|
|               US|     44.6962242564584|
|            Italy|   36.660851793517246|
|          Taiwan*|    30.03070210078223|
|          Vietnam|    18.31295693610047|
|           Russia|   16.464437211137984|
|   United Kingdom|   15.647920672439058|
|           Greece|     9.71600484451953|
|          Austria|    9.637770347748694|
|  New South Wales|    8.075024743816195|
|        Hong Kong|     7.88982069086734|
|           Turkey|    7.294280976090078|
|      Netherlands|    6.953956435839991|
|            Spain|    6.472320789791172|
|         Portugal|    6.272707391527799|
|         Victoria|    5.785826797368918|
|      New Zealand|    5.520522814150511|
|            Chile|    4.959192737

In [20]:
#Joining trendline coefficient with dataframe with main dataframe and ordering it
df_top_50_kmean = df_window.join(df_top_50, on = ["Zone"], how = 'inner').orderBy(['trendline_coefficient', 'date'],
                                                                                  ascending = [False, True])

In [21]:
df_top_50_kmean.show()

+-----+-----+-------+----------+-----+----------+---------------+---------------------+
| Zone|State|Country|      date|value|No_of_Days|daily_increases|trendline_coefficient|
+-----+-----+-------+----------+-----+----------+---------------+---------------------+
|Japan| null|  Japan|2020-01-22|    2|         0|              2|    94.19123188154933|
|Japan| null|  Japan|2020-01-23|    2|         1|              0|    94.19123188154933|
|Japan| null|  Japan|2020-01-24|    2|         2|              0|    94.19123188154933|
|Japan| null|  Japan|2020-01-25|    2|         3|              0|    94.19123188154933|
|Japan| null|  Japan|2020-01-26|    4|         4|              2|    94.19123188154933|
|Japan| null|  Japan|2020-01-27|    4|         5|              0|    94.19123188154933|
|Japan| null|  Japan|2020-01-28|    7|         6|              3|    94.19123188154933|
|Japan| null|  Japan|2020-01-29|    7|         7|              0|    94.19123188154933|
|Japan| null|  Japan|2020-01-30|

In [22]:
#preparing vector for Kmean-Clustering
vector_assembler2 = VectorAssembler(inputCols=["trendline_coefficient"], outputCol="kfeatures")

df_kassembled = vector_assembler2.transform(df_top_50_kmean)

df_kassembled.show()

+-----+-----+-------+----------+-----+----------+---------------+---------------------+-------------------+
| Zone|State|Country|      date|value|No_of_Days|daily_increases|trendline_coefficient|          kfeatures|
+-----+-----+-------+----------+-----+----------+---------------+---------------------+-------------------+
|Japan| null|  Japan|2020-01-22|    2|         0|              2|    94.19123188154933|[94.19123188154933]|
|Japan| null|  Japan|2020-01-23|    2|         1|              0|    94.19123188154933|[94.19123188154933]|
|Japan| null|  Japan|2020-01-24|    2|         2|              0|    94.19123188154933|[94.19123188154933]|
|Japan| null|  Japan|2020-01-25|    2|         3|              0|    94.19123188154933|[94.19123188154933]|
|Japan| null|  Japan|2020-01-26|    4|         4|              2|    94.19123188154933|[94.19123188154933]|
|Japan| null|  Japan|2020-01-27|    4|         5|              0|    94.19123188154933|[94.19123188154933]|
|Japan| null|  Japan|2020-01

In [23]:
#Implementing Kmeans Clustering
kmeans = KMeans(k = 4, seed = 42, featuresCol = "kfeatures", predictionCol = 'Clusters')

kmodel = kmeans.fit(df_kassembled)

kpredictions = kmodel.transform(df_kassembled)

kpredictions_zone_cluster = kpredictions.select("Zone", "trendline_coefficient", "Clusters").distinct()

In [24]:
kpredictions_zone_cluster.show()

+---------------+---------------------+--------+
|           Zone|trendline_coefficient|Clusters|
+---------------+---------------------+--------+
|        Taiwan*|    30.03070210078223|       3|
|       Slovenia|   1.5303530627799413|       0|
|        Finland|   2.4763042219535163|       0|
|         Israel|     4.22550203082354|       0|
|        Czechia|   1.6272593086684952|       0|
|       Malaysia|    4.109667476733129|       0|
|South Australia|   2.0267418044215217|       0|
|         Norway|   1.6022807593094555|       0|
|        Romania|   1.2300557576837645|       0|
|        Vietnam|    18.31295693610047|       3|
|         Serbia|   1.6107994032618604|       0|
|    Netherlands|    6.953956435839991|       0|
|         Cyprus|   0.9513468475987602|       0|
|          Italy|   36.660851793517246|       3|
|             US|     44.6962242564584|       2|
|   Korea, South|    72.67960297903623|       2|
|          Spain|    6.472320789791172|       0|
|       Victoria|   

In [25]:
end_time = time.time()

In [26]:
total_time = end_time-start_time

In [27]:
print(total_time)
print(start_time)
print(end_time)

891.1885912418365
1700402941.680053
1700403832.8686442


In [28]:
kpredictions_zone_cluster.write.csv("Answer to Question 3", header=True, mode="overwrite") 

In [29]:
kpredictions.select("Clusters").distinct().show()

+--------+
|Clusters|
+--------+
|       1|
|       3|
|       2|
|       0|
+--------+



Per month implementation of Linear Regression. Computationally expensive, hence abandoned.

In [50]:
df_window2 = df_window.withColumn("year_month", F.col("date").substr(1,7))

In [51]:
df_window2.show()

+-----+-----------+---------+-----------+----------+-----+----------+---------------+----------+
|State|    Country|Continent|       Zone|      date|value|No_of_Days|daily_increases|year_month|
+-----+-----------+---------+-----------+----------+-----+----------+---------------+----------+
| null|Afghanistan|     Asia|Afghanistan|2020-01-22|    0|         0|              0|   2020-01|
| null|Afghanistan|     Asia|Afghanistan|2020-01-23|    0|         1|              0|   2020-01|
| null|Afghanistan|     Asia|Afghanistan|2020-01-24|    0|         2|              0|   2020-01|
| null|Afghanistan|     Asia|Afghanistan|2020-01-25|    0|         3|              0|   2020-01|
| null|Afghanistan|     Asia|Afghanistan|2020-01-26|    0|         4|              0|   2020-01|
| null|Afghanistan|     Asia|Afghanistan|2020-01-27|    0|         5|              0|   2020-01|
| null|Afghanistan|     Asia|Afghanistan|2020-01-28|    0|         6|              0|   2020-01|
| null|Afghanistan|     Asia|A

In [47]:
df_month = df_window.select(F.col("date").substr(1, 7).alias("year_month")).distinct()

In [49]:
#Extracting Months
df_month = df_window.select(F.col("date").substr(1, 7).alias("year_month")).distinct()

months = df_month.collect()
print(months)

[Row(year_month='2020-06'), Row(year_month='2020-12'), Row(year_month='2020-02'), Row(year_month='2021-07'), Row(year_month='2022-04'), Row(year_month='2021-04'), Row(year_month='2022-06'), Row(year_month='2021-06'), Row(year_month='2022-02'), Row(year_month='2022-08'), Row(year_month='2020-05'), Row(year_month='2021-11'), Row(year_month='2023-03'), Row(year_month='2021-03'), Row(year_month='2021-02'), Row(year_month='2022-07'), Row(year_month='2023-02'), Row(year_month='2021-10'), Row(year_month='2021-12'), Row(year_month='2020-11'), Row(year_month='2022-11'), Row(year_month='2021-05'), Row(year_month='2022-10'), Row(year_month='2020-07'), Row(year_month='2023-01'), Row(year_month='2022-09'), Row(year_month='2020-03'), Row(year_month='2022-03'), Row(year_month='2022-05'), Row(year_month='2021-08'), Row(year_month='2020-01'), Row(year_month='2020-09'), Row(year_month='2022-12'), Row(year_month='2020-08'), Row(year_month='2021-01'), Row(year_month='2020-04'), Row(year_month='2020-10'), 

In [ ]:
result2 = []

In [55]:
for month in months:
    for zone in zones:
        df_zone_month = df_window2.filter((df_window2['Zone'] == zone[0]) & (df_window2['year_month'] == month[0])).select('No_of_Days','daily_increases')
        df_zone_month = df_zone.withColumnRenamed('daily_increases','label')
        df_zone_month_vector = vector_assembler1.transform(df_zone_month)
        model = lr.fit(df_zone_month_vector)
        trendline_coefficient = float(model.coefficients[0])
        result2.append([zone[0], month[0], trendline_coefficient])

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "C:\spark\spark-3.4.1-bin-hadoop3\python\lib\py4j-0.10.9.7-src.zip\py4j\java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "C:\spark\spark-3.4.1-bin-hadoop3\python\lib\py4j-0.10.9.7-src.zip\py4j\clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "C:\Users\offic\anaconda3\envs\pyspark_env\lib\socket.py", line 669, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [56]:
print(result)

[['Chad', -0.006532846425561239], ['Paraguay', 0.2240196760550598], ['Russia', 16.464437211137984], ['Manitoba', 0.05721877512974767], ['Anguilla', 0.005211799384200336], ['Yemen', -0.008412905543923103], ['Korea, North', 2.1858068936684977e-06], ['Guangdong', 0.3686350147042123], ['Senegal', -0.07072891806126784], ['Cabo Verde', -0.01094125730762666], ['Sweden', 0.5866950480834147], ['Kiribati', 0.011824797419899256], ['Hunan', 0.01974093013090765], ['Shanxi', 0.024976339443981508], ['Guyana', 0.03526536643943595], ['Burma', -0.0968252183484471], ['Eritrea', -0.003273036885844923], ['Philippines', 0.40715207426805927], ['Tibet', 0.005224777612631496], ['Jersey', 0.09750150057250453], ['Djibouti', -0.022944985522725926], ['Tonga', 0.037288033385500186], ['Malaysia', 4.109667476733129], ['Singapore', 4.460320854593922], ['Fiji', 0.021016043083282452], ['Hubei', -0.28688951739408874], ['Turkey', 7.294280976090078], ['Yukon', 0.006095460037294371], ['Tianjin', 0.010284623237448089], ['Mal

In [61]:
result2 = result[285:]

In [62]:
print(result2)

[['Chad', '2020-06', 0.1277311295513923], ['Paraguay', '2020-06', 0.1277311295513923], ['Russia', '2020-06', 0.1277311295513923], ['Manitoba', '2020-06', 0.1277311295513923], ['Anguilla', '2020-06', 0.1277311295513923], ['Yemen', '2020-06', 0.1277311295513923], ['Korea, North', '2020-06', 0.1277311295513923], ['Guangdong', '2020-06', 0.1277311295513923], ['Senegal', '2020-06', 0.1277311295513923], ['Cabo Verde', '2020-06', 0.1277311295513923], ['Sweden', '2020-06', 0.1277311295513923], ['Kiribati', '2020-06', 0.1277311295513923], ['Hunan', '2020-06', 0.1277311295513923], ['Shanxi', '2020-06', 0.1277311295513923], ['Guyana', '2020-06', 0.1277311295513923], ['Burma', '2020-06', 0.1277311295513923], ['Eritrea', '2020-06', 0.1277311295513923], ['Philippines', '2020-06', 0.1277311295513923], ['Tibet', '2020-06', 0.1277311295513923], ['Jersey', '2020-06', 0.1277311295513923], ['Djibouti', '2020-06', 0.1277311295513923], ['Tonga', '2020-06', 0.1277311295513923], ['Malaysia', '2020-06', 0.1277

In [64]:
R2 = Row("Zone","Month","trendline_coefficient")
df_result2 = spark.createDataFrame([R2(*i) for i in result2])
df_result2 = df_result2.orderBy(["Month","trendline_coefficient"], ascending = [True, False])

In [69]:
df_result2.show(500)

+--------------------+-------+---------------------+
|                Zone|  Month|trendline_coefficient|
+--------------------+-------+---------------------+
|               Samoa|2020-01|   0.1277311295513923|
|                Chad|2020-01|   0.1277311295513923|
|            Paraguay|2020-01|   0.1277311295513923|
|              Russia|2020-01|   0.1277311295513923|
|            Manitoba|2020-01|   0.1277311295513923|
|            Anguilla|2020-01|   0.1277311295513923|
|               Yemen|2020-01|   0.1277311295513923|
|        Korea, North|2020-01|   0.1277311295513923|
|           Guangdong|2020-01|   0.1277311295513923|
|             Senegal|2020-01|   0.1277311295513923|
|          Cabo Verde|2020-01|   0.1277311295513923|
|              Sweden|2020-01|   0.1277311295513923|
|            Kiribati|2020-01|   0.1277311295513923|
|               Hunan|2020-01|   0.1277311295513923|
|              Shanxi|2020-01|   0.1277311295513923|
|              Guyana|2020-01|   0.12773112955

In [70]:
windowspecs = Window.partitionBy("Month").orderBy("trendline_coefficient")

In [72]:
df_result2_window = df_result2.withColumn("row_number", F.row_number().over(windowspecs))

In [73]:
df_result2_window2 = df_result2_window.filter(df_result2_window.row_number <= 50).orderBy(["Month","trendline_coefficient"], 
                                                                                          ascending = [True, False])

In [76]:
df_result2_window2.show(1000)

+--------------------+-------+---------------------+----------+
|                Zone|  Month|trendline_coefficient|row_number|
+--------------------+-------+---------------------+----------+
|                Chad|2020-01|   0.1277311295513923|         1|
|            Paraguay|2020-01|   0.1277311295513923|         2|
|              Russia|2020-01|   0.1277311295513923|         3|
|            Manitoba|2020-01|   0.1277311295513923|         4|
|            Anguilla|2020-01|   0.1277311295513923|         5|
|               Yemen|2020-01|   0.1277311295513923|         6|
|        Korea, North|2020-01|   0.1277311295513923|         7|
|           Guangdong|2020-01|   0.1277311295513923|         8|
|             Senegal|2020-01|   0.1277311295513923|         9|
|          Cabo Verde|2020-01|   0.1277311295513923|        10|
|              Sweden|2020-01|   0.1277311295513923|        11|
|            Kiribati|2020-01|   0.1277311295513923|        12|
|               Hunan|2020-01|   0.12773

In [77]:
df_result2_window2.write.csv("Question3 part 1", header=True, mode="overwrite")